In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
from matplotlib import pyplot as plt
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

import imageio
from PIL import Image

In [2]:
# Load GT detections
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

# Load computed detections
detection_path = 'retinanet_101_detections_trained.pkl'
data_path = '../../AICity_data/train/S03/c010/'

with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

# Load video frames
video_path = '../../AICity_data/train/S03/c010/vdo.avi' #UPDATE PATH WHEN PUSHING
vidcap = cv2.VideoCapture(video_path)
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

## Overlapping boxes approach

In [4]:
def update_track3(detections_pd, previous_detections_pd, tolerance = 0.5, imgs = None):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)

    previous_tracks = []

    for index, previous_detection in previous_detections_pd.iterrows():
        length, _ = detections_pd.shape

        # Calculating IoUs
        IoUlist = []
        for i in range(length):
            IoU = previous_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)

        indexMax = IoUlist.index(max(IoUlist))

        # Updating detection tracks based on best IoU matches
        if max(IoUlist) > tolerance and detections_pd.at[indexMax, 'updated'] != True:
            detections_pd.at[indexMax, 'track'] = previous_detection['track']
            detections_pd.at[indexMax, 'colour'] = previous_detection['colour']
            for line in previous_detection['line']:
                if line != detections_pd.at[indexMax,'line'][0]:
                    detections_pd.at[indexMax,'line'].append(line)
            detections_pd.at[indexMax, 'updated'] = True

        previous_tracks.append(previous_detection['track'])

    # Create new tracks for unmatched detections
    idxs = detections_pd.index[detections_pd['updated'] == False].tolist()
    track_check = 1
    for ind in idxs:
        while track_check in previous_tracks:
            track_check += 1
        detections_pd.at[ind, 'track'] = track_check
        previous_tracks.append(track_check)
            
    return detections_pd

In [5]:
detection_history = []

# Get the first frame
detections_prev_pd = get_detection_dataframe(all_detections['0'], firstFrame = True)

# Pre-process the first frame if needed
detections_prev_pd = remove_overlaps(detections_prev_pd, 0.9)
detection_history.append(detections_prev_pd)

vidcap = cv2.VideoCapture(video_path)
_, frame_0 = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, image = vidcap.read()
    
    detections_pd = get_detection_dataframe(all_detections[str(frame)])
    detections_pd = remove_overlaps(detections_pd, 0.9)
    detections_pd = update_track3(detections_pd, detections_prev_pd, tolerance=0.5)
    detection_history.append(detections_pd)
    detections_prev_pd = detections_pd

pkl.dump(detection_history, open('tracking_history3.pkl', 'wb'))

100%|██████████| 2139/2139 [02:53<00:00, 12.33it/s]


In [6]:
detections = {}
with open('tracking_history3.pkl','rb') as openFile:
    detections = pkl.load(openFile)

## Animation generation

In [7]:
gif = True
initial_gif_frame = 600
final_gif_frame = 650
scaling = 0.5

ims = []

vidcap = cv2.VideoCapture(video_path)
vidcap.set(1,initial_gif_frame)

if gif:
    _, initial_image = vidcap.read()

    for i in range(initial_gif_frame, final_gif_frame):
        _, image = vidcap.read()
        for index, row in detections[i].iterrows():
            image = drawTrackingOnImage(image, row['bbox'], track=row['track'], line=row['line'], colour=row['colour'])
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (int(image.shape[1]*scaling), int(image.shape[0]*scaling)), interpolation = cv2.INTER_AREA)

        ims.append(image)

    imageio.mimsave('test_animation.gif', ims, fps=20, duration = 0.1)

## Evaluation

In [8]:
total_frames = 2141
initial_frame = 535

In [9]:
acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    acc, frame_id = tracking_acc(frame, gt_detect, detections, acc)

display_metrics(acc)

100%|██████████| 1605/1605 [01:26<00:00, 18.53it/s]


     num_frames  precision  recall       idp       idr     idf1
acc        1605   0.593536     1.0  0.570797  0.961689  0.71639


## IGNORE EVERYTHING AFTER THIS POINT

In [9]:
ims = []
detection_history = []
gif = False

# if gif:
#     fig, ax = plt.subplots()
#     plt.axis('off')

# Get the first frame
detections_pd = get_detection_dataframe(all_detections['0'])

# Pre-process the first frame if needed
detections_pd = remove_overlaps(detections_pd, 0.9)
detection_history.append(detections_pd)

_, past_frame = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, image = vidcap.read()
    # vidcap.set(1,'frame')
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    next_detections_pd = remove_overlaps(next_detections_pd, 0.9)
    
    detections_pd = update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=[past_frame, image])
    detection_history.append(detections_pd)
    past_frame = image

    # if gif:
    #     for index, row in detections_pd.iterrows():
    #         image = drawTrackingOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])
    #     im = ax.imshow(image, animated=True)
    #     ims.append([im])

# if gif:
#     ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
#     ani.save('all_detects' + ".gif", writer=animation.PillowWriter(fps=24))
    
pkl.dump(detection_history, open('tracking_history.pkl', 'wb'))

  0%|          | 0/2139 [00:00<?, ?it/s]


NameError: name 'update_track2' is not defined

In [ ]:
# def update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=None):
#     detections_pd['updated'] = False
#     detections_pd = detections_pd.reset_index(drop=True)
    
#     # Loop each new detection
#     for index, next_detection in next_detections_pd.iterrows():
#         length, _ = detections_pd.shape
        
#         # Find overlaps with max IoU and update if found
#         IoUlist = []
#         for i in range(length):
#             IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
#             IoUlist.append(IoU)
            
#         indexMax = IoUlist.index(max(IoUlist))
            
#         if max(IoUlist) > tolerance and detections_pd.at[indexMax,'updated'] != True:
#             detections_pd.at[indexMax,'detection'] = next_detection['detection']
#             detections_pd.at[indexMax,'bbox'] = next_detection['bbox']
#             detections_pd.at[indexMax,'size'] = next_detection['size']
#             detections_pd.at[indexMax,'line'].append(next_detection['line'][0])
#             detections_pd.at[indexMax,'updated'] = True
#             detections_pd.at[indexMax, 'opt_flow'] = 0
#             next_detections_pd.at[index, 'updated'] = True
    


#     # If frame images "imgs" are provided, use optical flow approach to improve tracking
#     if imgs is not None:
#         idxs = detections_pd.index[detections_pd['updated'] == False].tolist() #Non-updated detections
#         for ind in idxs:
#             if detections_pd.at[ind, 'opt_flow'] < 1: # After _ consecutive frames using OF, remove detection
#                 box = detections_pd.at[ind, 'bbox']
#                 block = imgs[0][int(box[1]):int(box[3]), int(box[0]):int(box[2])]

#                 #Compute optical flow of the not updated tracking bounding box
#                 of = compute_block_of(block, box, imgs[1], 'backward', 10)
                
#                 #Update bounding box
#                 if len(of) > 0:
#                     if (of[0] != 0.0 or of[1] != 0.0):
#                         newBox = [box[0]+of[0], box[1]+of[1], box[2]+of[0], box[3]+of[1]]
#                         detections_pd.at[ind, 'bbox'] = newBox
#                         detections_pd.at[ind, 'updated'] = True # NEED TO DOUBLE CHECK THIS
#                         detections_pd.at[ind, 'detection'].updateBBox(newBox)
#                         detections_pd.at[ind, 'line'].append(get_box_center(newBox))
#                         detections_pd.at[ind, 'opt_flow'] = detections_pd.at[ind, 'opt_flow'] + 1


#     # Drop detections no longer exist
#     detections_pd = detections_pd[detections_pd['updated'] == True]
                
#     # Start tracking new detections
#     counter = 0
#     if any(next_detections_pd['updated'] == False):
#         new_pd = next_detections_pd[next_detections_pd['updated'] == False]
#         new_pd = new_pd.reset_index(drop=True)
        
#         # Generate new track number      
#         for i in range(len(new_pd)):
#             while counter in detections_pd['track'].tolist():
#                 counter = counter + 1
#             new_pd.at[i, 'track'] = counter
#             counter = counter + 1

#         # Add new tracks
#         detections_pd = pd.concat([detections_pd, new_pd])
            
#     detections_pd = detections_pd.reset_index(drop=True)
#     return detections_pd